In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from datetime import datetime
from tqdm import tqdm
from tqdm.contrib.concurrent import process_map
from tqdm.contrib import tmap

# Enable tqdm for Pandas
tqdm.pandas()

/Users/esmirmesic/opt/anaconda3/envs/bem114/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
crsp_data = pd.read_csv("data/cleaned_crsp.csv")
crsp_data['date'] = pd.to_datetime(crsp_data['date'])
crsp_data['RET'] = crsp_data['RET'].str.replace('C', '')
crsp_data['RET'] = pd.to_numeric(crsp_data['RET'], errors='coerce')
crsp_data['date'] = pd.to_datetime(crsp_data['date'], format='%Y-%m-%d')

In [3]:
# Calculate market value of equity (ME) for each stock
# crsp_data['mkt_cap'] = np.abs(crsp_data['PRC']) * crsp_data['SHROUT']

start_date = '1926-01-01'
end_date = '2020-12-31'

# # get cumulative returns
# def get_cum(stock):
#     cum_ret = []
#     stock_yr = pd.date_range(start_date, periods=2020-1926+1, freq="Y")
#     for y in stock_yr:
#         ind = stock['date'].dt.year == y
#         cum_ret.append(sum(stock.loc[ind]['RET']))
#     stock['cum_ret'] = cum_ret
#     return cum_ret

crsp_data['cum_ret'] = crsp_data.groupby('PERMNO')['RET'].rolling(window=11).progress_apply(lambda x: np.prod(1 + x) - 1, raw=True).reset_index(0, drop=True)
# Groupby PERMNO and year, and calculate cumulative monthly returns using the function


# # Define a function to assign deciles based on market cap
def assign_deciles(data):
    # Check if there are any non-NaN values in the 'cum_ret' column
    if pd.notna(data['cum_ret']).any():
        data['decile'] = pd.qcut(data['cum_ret'], 10, labels=False) + 1
    else:
        # Set decile to NaN if there are no valid values in 'cum_ret'
        data['decile'] = np.nan
    return data

crsp_data = crsp_data.groupby('date').progress_apply(assign_deciles).reset_index(drop=True)

# get equal- and value-weighted portfolios
def calculate_portfolio_returns(data):
    ew_ret = data['RET'].mean()
    vw_ret = np.average(data['RET'], weights=data['cum_ret'])
    return pd.Series({'ew_ret': ew_ret, 'vw_ret': vw_ret})

# Group the data by date and decile and calculate the returns for each group
portfolio_returns = crsp_data.groupby(['date', 'decile']).apply(calculate_portfolio_returns).reset_index()

# Pivot the data to get a wide format with deciles as columns
ew_returns = portfolio_returns.pivot_table(values='ew_ret', index='date', columns='decile')
vw_returns = portfolio_returns.pivot_table(values='vw_ret', index='date', columns='decile')


3279165it [00:18, 177862.31it/s]
100%|██████████| 1141/1141 [00:02<00:00, 486.90it/s] 


In [12]:
# Calculate mean returns for each decile
mean_ew_returns = ew_returns.mean()
mean_vw_returns = vw_returns.mean()

# Check if the returns are monotonic
is_monotonic_ew = mean_ew_returns.is_monotonic_decreasing
is_monotonic_vw = mean_vw_returns.is_monotonic_decreasing

print("Mean equal-weighted returns:")
print(mean_ew_returns)
print("Is monotonic:", is_monotonic_ew)
print("\nMean value-weighted returns:")
print(mean_vw_returns)
print("Is monotonic:", is_monotonic_vw)

Mean equal-weighted returns:
1.0      -0.065779
2.0      -0.022565
3.0      -0.009115
4.0       0.000376
5.0       0.008129
6.0       0.015479
7.0       0.022623
8.0       0.033033
9.0       0.049008
10.0      0.094732
const     1.000000
Mkt-RF    0.005756
SMB       0.002439
HML       0.002606
RMW       0.002399
CMA       0.001809
RF        0.003776
dtype: float64
Is monotonic: False

Mean value-weighted returns:
1.0      -0.053345
2.0      -0.026561
3.0      -0.013505
4.0      -0.004174
5.0       0.003916
6.0       0.010849
7.0       0.017737
8.0       0.027221
9.0       0.040365
10.0      0.073702
const     1.000000
Mkt-RF    0.005756
SMB       0.002439
HML       0.002606
RMW       0.002399
CMA       0.001809
RF        0.003776
dtype: float64
Is monotonic: False


In [28]:
def form_wml_portfolios(group):
    winners = group[group['decile'] == 10.0]
    losers = group[group['decile'] == 1.0]
    
    # Calculate equal-weighted average returns for winners and losers
    winners_ret_ew = winners['RET'].mean()
    losers_ret_ew = losers['RET'].mean()

    vw_winners_ret = np.average(winners['RET'], weights=winners['cum_ret']) if winners['cum_ret'].sum() != 0 else np.nan
    vw_losers_ret = np.average(losers['RET'], weights=losers['cum_ret']) if losers['cum_ret'].sum() != 0 else np.nan
    
    
    # Calculate winners-minus-losers return
    wml_ret_ew = winners_ret_ew - losers_ret_ew
    wml_ret_vw = vw_winners_ret - vw_losers_ret

    return pd.Series({
        'ew_wml_ret': wml_ret_ew,
        'vw_wml_ret': wml_ret_vw
    })

wml_returns = crsp_data.groupby('date').apply(form_wml_portfolios)

# Extract equal-weighted and value-weighted WML returns
ew_returns = wml_returns['ew_wml_ret']
vw_returns = wml_returns['vw_wml_ret']

# Print the results
print("Equal-Weighted WML Portfolio Returns:")
print(ew_returns)
print("\nValue-Weighted WML Portfolio Returns:")
print(vw_returns)

Equal-Weighted WML Portfolio Returns:
date
1925-12-31 00:00:00         NaN
1926-01-30 00:00:00         NaN
1926-02-27 00:00:00         NaN
1926-03-31 00:00:00         NaN
1926-04-30 00:00:00         NaN
                         ...   
2020-09-30 00:00:00    0.229146
2020-10-30 00:00:00    0.129522
2020-11-30 00:00:00    0.132529
2020-12-31 00:00:00    0.232340
const                  1.000000
Name: ew_wml_ret, Length: 1142, dtype: float64

Value-Weighted WML Portfolio Returns:
date
1925-12-31 00:00:00         NaN
1926-01-30 00:00:00         NaN
1926-02-27 00:00:00         NaN
1926-03-31 00:00:00         NaN
1926-04-30 00:00:00         NaN
                         ...   
2020-09-30 00:00:00    0.279546
2020-10-30 00:00:00    0.123934
2020-11-30 00:00:00    0.259646
2020-12-31 00:00:00    0.270567
const                  1.000000
Name: vw_wml_ret, Length: 1142, dtype: float64


In [18]:
import pandas_datareader as pdr

start_date = '1926-01-01'
end_date = '2020-12-31'

# Download Fama-French 3-factor data
ff3_factors = pdr.get_data_famafrench('F-F_Research_Data_Factors', start=start_date, end=end_date)[0]
ff3_factors = ff3_factors / 100  # Convert to decimal
ff3_factors.index = ff3_factors.index.to_timestamp('M')  # Convert index to monthly-end dates

# FF5 - FIX DATA SOURCE
ff5_factors = pdr.get_data_famafrench('F-F_Research_Data_5_Factors_2x3', start=start_date, end=end_date)[0]
ff5_factors = ff5_factors / 100  # Convert to decimal
ff5_factors.index = ff5_factors.index.to_timestamp('M')  # Convert index to monthly-end dates

/var/folders/sg/4dp480wd1cjd288xvby34rpr0000gn/T/ipykernel_10802/2391014569.py:7: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  ff3_factors = pdr.get_data_famafrench('F-F_Research_Data_Factors', start=start_date, end=end_date)[0]
/var/folders/sg/4dp480wd1cjd288xvby34rpr0000gn/T/ipykernel_10802/2391014569.py:7: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  ff3_factors = pdr.get_data_famafrench('F-F_Research_Data_Factors', start=start_date, end=end_date)[0]
/var/folders/sg/4dp480wd1cjd288xvby34rpr0000gn/T/ipykernel_10802/2391014569.py:12: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your dat

In [29]:
def estimate_models(returns, factors, factors5):
    # Add a constant to the factors for regression
    factors = sm.add_constant(factors)

    # Estimate the CAPM model
    capm_model = sm.OLS(returns, factors[['const', 'Mkt-RF']]).fit()

    # Estimate the FF3 model
    ff3_model = sm.OLS(returns, factors).fit()

    # estimate FF5
    ff5_model = sm.OLS(returns, factors5).fit()

    return capm_model.params, ff3_model.params, ff5_model.params

# Merge the factor data with the portfolio returns
# Add a constant column to the returns DataFrames
ew_returns['const'] = 1
vw_returns['const'] = 1

# Merge the factor data with the portfolio returns
# ew_returns = ew_returns.merge(ff3_factors, left_index=True, right_index=True, suffixes=('', '_y'))
# vw_returns = vw_returns.merge(ff3_factors, left_index=True, right_index=True, suffixes=('', '_y'))

# Merge the ff5 data with the portfolio returns
ew_returns = ew_returns.merge(ff5_factors, left_index=True, right_index=True, suffixes=('', '_y'))
vw_returns = vw_returns.merge(ff5_factors, left_index=True, right_index=True, suffixes=('', '_y'))


# Calculate the CAPM and FF3 model parameters for each decile
ew_results = pd.DataFrame()
vw_results = pd.DataFrame()

for decile in range(1, 11):
    ew_capm_params, ew_ff3_params = estimate_models(ew_returns[decile], ew_returns[['const', 'Mkt-RF', 'SMB', 'HML']], ff5_factors)
    vw_capm_params, vw_ff3_params = estimate_models(vw_returns[decile], vw_returns[['const', 'Mkt-RF', 'SMB', 'HML']], ff5_factors)

    ew_results = pd.concat([ew_results, pd.concat([ew_capm_params, ew_ff3_params], keys=['CAPM', 'FF3', 'FF5'])], axis=1)
    vw_results = pd.concat([vw_results, pd.concat([vw_capm_params, vw_ff3_params], keys=['CAPM', 'FF3', 'FF5'])], axis=1)

ew_results.columns = range(1, 11)
vw_results.columns = range(1, 11)


print("Equal-weighted portfolio results:")
print(ew_results)

print("\nValue-weighted portfolio results:")
print(vw_results)


AttributeError: 'Series' object has no attribute 'merge'

In [ ]:
ew_results

1         2         3         4         5         6    
CAPM const  -0.015925  0.001009  0.003584  0.005511  0.007938  0.008830  \
     Mkt-RF  1.656576  1.599239  1.456843  1.432393  1.399146  1.340762   
FF3  const  -0.019065 -0.001587  0.001684  0.003979  0.006661  0.007767   
     Mkt-RF  1.134937  1.152042  1.108285  1.135423  1.140081  1.122068   
     SMB     1.621889  1.440500  1.187582  1.058033  0.954193  0.813348   
     HML     0.967517  0.750562  0.482891  0.338609  0.246224  0.195488   

                   7         8         9         10  
CAPM const   0.009166  0.009423  0.008992  0.008153  
     Mkt-RF  1.276111  1.212673  1.138387  0.989791  
FF3  const   0.008404  0.008890  0.008614  0.008164  
     Mkt-RF  1.112478  1.091643  1.063974  0.993081  
     SMB     0.626072  0.478973  0.267695 -0.014604  
     HML     0.118697  0.062730  0.080782  0.000789

In [ ]:
vw_results

1         2         3         4         5         6    
CAPM const  -0.009810  0.001199  0.003483  0.005589  0.007891  0.008733  \
     Mkt-RF  1.628531  1.589552  1.447115  1.425369  1.392176  1.327447   
FF3  const  -0.012750 -0.001376  0.001603  0.004058  0.006621  0.007703   
     Mkt-RF  1.131423  1.146148  1.102706  1.129894  1.135195  1.114320   
     SMB     1.573197  1.427876  1.172379  1.049061  0.944606  0.796081   
     HML     0.878563  0.744832  0.478828  0.342646  0.247255  0.185094   

                   7         8         9         10  
CAPM const   0.009095  0.009279  0.008734  0.008034  
     Mkt-RF  1.267836  1.203432  1.125723  0.934020  
FF3  const   0.008348  0.008767  0.008370  0.008203  
     Mkt-RF  1.107443  1.087439  1.055901  0.968997  
     SMB     0.614235  0.459134  0.246864 -0.130755  
     HML     0.115457  0.059971  0.082598 -0.030205

In [ ]:
# Set the date ranges
post_ff_paper_start = '1993-01-01'
post_ff_paper_end = '2001-12-31'
post_dotcom_start = '2002-01-01'

# Create the subsets
ew_returns_post_ff = ew_returns.loc[(ew_returns.index >= post_ff_paper_start) & (ew_returns.index <= post_ff_paper_end)]
vw_returns_post_ff = vw_returns.loc[(vw_returns.index >= post_ff_paper_start) & (vw_returns.index <= post_ff_paper_end)]

ew_returns_post_dotcom = ew_returns.loc[ew_returns.index >= post_dotcom_start]
vw_returns_post_dotcom = vw_returns.loc[vw_returns.index >= post_dotcom_start]



In [ ]:
def calculate_statistics(returns):
    mean = returns.mean()
    volatility = returns.std()
    sharpe_ratio = mean / volatility
    return mean, volatility, sharpe_ratio

# Post Fama French 1992 paper
ew_mean_post_ff, ew_vol_post_ff, ew_sharpe_post_ff = calculate_statistics(ew_returns_post_ff.iloc[:, -1] - ew_returns_post_ff.iloc[:, 0])
vw_mean_post_ff, vw_vol_post_ff, vw_sharpe_post_ff = calculate_statistics(vw_returns_post_ff.iloc[:, -1] - vw_returns_post_ff.iloc[:, 0])

# Post Dot-Com Bubble
ew_mean_post_dotcom, ew_vol_post_dotcom, ew_sharpe_post_dotcom = calculate_statistics(ew_returns_post_dotcom.iloc[:, -1] - ew_returns_post_dotcom.iloc[:, 0])
vw_mean_post_dotcom, vw_vol_post_dotcom, vw_sharpe_post_dotcom = calculate_statistics(vw_returns_post_dotcom.iloc[:, -1] - vw_returns_post_dotcom.iloc[:, 0])


In [ ]:
print("Post Fama French 1992 paper:")
print(f"Equal-weighted SMB portfolio - Mean: {ew_mean_post_ff}, Volatility: {ew_vol_post_ff}, Sharpe Ratio: {ew_sharpe_post_ff}")
print(f"Value-weighted SMB portfolio - Mean: {vw_mean_post_ff}, Volatility: {vw_vol_post_ff}, Sharpe Ratio: {vw_sharpe_post_ff}")

print("\nPost Dot-Com Bubble:")
print(f"Equal-weighted SMB portfolio - Mean: {ew_mean_post_dotcom}, Volatility: {ew_vol_post_dotcom}, Sharpe Ratio: {ew_sharpe_post_dotcom}")
print(f"Value-weighted SMB portfolio - Mean: {vw_mean_post_dotcom}, Volatility: {vw_vol_post_dotcom}, Sharpe Ratio: {vw_sharpe_post_dotcom}")


Post Fama French 1992 paper:
Equal-weighted SMB portfolio - Mean: 0.028723789029534253, Volatility: 0.09651448361997186, Sharpe Ratio: 0.2976111766046936
Value-weighted SMB portfolio - Mean: 0.015741403229977663, Volatility: 0.08941032442062882, Sharpe Ratio: 0.17605800372586272

Post Dot-Com Bubble:
Equal-weighted SMB portfolio - Mean: 0.02286196226482209, Volatility: 0.08119419005368132, Sharpe Ratio: 0.28157140615242254
Value-weighted SMB portfolio - Mean: 0.012345695771850811, Volatility: 0.07930922612602084, Sharpe Ratio: 0.1556653163181007
